# Statement

After some discussion with Jörg about the best implementation of the symmetry functions it may pay to take a top-down view, and calculate the symmetry funcitons on a per-triple basis. Firstly, constructing the $r^2$ matrix implies that an $f_c(r^2)$ matrix can be computed and stored only once, reducing the frequency of computation drastically

In [1]:
using Distributed
using MachineLearningPotential
using BenchmarkTools
using StaticArrays

Loading the atomic parameters and symmetry functions

In [2]:
atoms = [[0.0000006584,       -0.0000019175,        0.0000000505],
[-0.0000005810,       -0.0000004871,        0.6678432175],
[0.1845874248,       -0.5681026047,        0.2986701538],
[-0.4832557457,       -0.3511072166,        0.2986684497],
[-0.4832557570,        0.3511046452,        0.2986669456],
[0.1845874064,        0.5681000550,        0.2986677202],
[0.5973371920,       -0.0000012681,        0.2986697030],
[-0.1845860897,       -0.5681038901,       -0.2986676192],
[-0.5973358752,       -0.0000025669,       -0.2986696020],
[-0.1845861081,        0.5680987696,       -0.2986700528],
[0.4832570624,        0.3511033815,       -0.2986683486],
[0.4832570738,       -0.3511084803,       -0.2986668445],
[0.0000018978,       -0.0000033480,       -0.6678431165],
[-0.0000017969,        0.0000009162,        1.3230014650],
[0.1871182835,       -0.5758942175,        0.9797717078],
[-0.4898861924,       -0.3559221410,       0.9797699802],
[-0.4898862039,        0.3559224872,        0.9797684555],
[0.1871182648,        0.5758945856,        0.9797692407],
[0.6055300485,        0.0000001908,        0.9797712507],
[0.7926501864,       -0.5758950093,        0.6055339635],
[0.3656681761,       -1.1254128670,        0.5916673591],
[-0.3027660545,       -0.9318173412,        0.6055326929],
[-0.9573332453,       -0.6955436707,        0.5916639831],
[-0.9797705418,       -0.0000006364,        0.6055294407],
[-0.9573332679,        0.6955423392,        0.5916610035],
[-0.3027660847,        0.9318160902,        0.6055287012],
[0.3656681396,        1.1254115783,        0.5916625380],
[0.7926501677,        0.5758937939,        0.6055314964],
[1.1833279992,       -0.0000006311,        0.5916664660],
[0.6770051458,       -0.9318186223,        0.0000033028],
[0.0000006771,       -1.1517907207,        0.0000025175],
[-0.6770037988,       -0.9318186442,        0.0000007900],
[-1.0954155825,       -0.3559242494,       -0.0000012200],
[-1.0954155940,        0.3559203788,       -0.0000027447],
[-0.6770038290,        0.9318147872,       -0.0000032017],
[0.0000006397,        1.1517868856,       -0.0000024165],
[0.6770051155,        0.9318148091,       -0.0000006889],
[1.0954168993,        0.3559204143,        0.0000013211],
[1.0954169108,       -0.3559242139,        0.0000028458],
[0.3027674014,       -0.9318199253,       -0.6055286002],
[-0.3656668229,       -1.1254154134,       -0.5916624370],
[-0.7926488510,       -0.5758976290,       -0.6055313954],
[-1.1833266824,       -0.0000032040,       -0.5916663649],
[-0.7926488697,        0.5758911742,       -0.6055338624],
[-0.3656668594,        1.1254090319,       -0.5916672580],
[0.3027673712,        0.9318135061,       -0.6055325919],
[0.9573345621,        0.6955398357,       -0.5916638820],
[0.9797718586,       -0.0000031986,       -0.6055293396],
[0.9573345846,       -0.6955461743,       -0.5916609025],
[-0.1871169480,       -0.5758984207,       -0.9797691397],
[-0.6055287318,       -0.0000040259,       -0.9797711497],
[-0.1871169667,        0.5758903824,       -0.9797716067],
[0.4898875091,        0.3559183059,       -0.9797698792],
[0.4898875207,       -0.3559263223,       -0.9797683545],
[0.0000031136,       -0.0000047513,       -1.3230013639]]*18.8973*0.36258

positions = [SVector{3}(p[i] for i in 1:3) for p in atoms]
dis2mat = get_distance2_mat(positions)
X = [ 1    1              0.001   0.000  11.338
 1    0              0.001   0.000  11.338
 1    1              0.020   0.000  11.338
 1    0              0.020   0.000  11.338
 1    1              0.035   0.000  11.338
 1    0              0.035   0.000  11.338
 1    1              0.100   0.000  11.338
 1    0              0.100   0.000  11.338
 1    1              0.400   0.000  11.338
 1    0              0.400   0.000  11.338]

radsymmvec = []

for row in eachrow(X)
    symmfunc = RadialType2{Float64}(row[3],row[5],[row[1],row[2]])
    push!(radsymmvec,symmfunc)
end

V = [[0.0001,1,1,11.338],[0.0001,-1,2,11.338],[0.003,-1,1,11.338],[0.003,-1,2,11.338],[0.008,-1,1,11.338],[0.008,-1,2,11.228],[0.008,1,2,11.338],[0.015,1,1,11.338],[0.015,-1,2,11.338],[0.015,-1,4,11.338],[0.015,-1,16,11.338],[0.025,-1,1,11.338],[0.025,1,1,11.338],[0.025,1,2,11.338],[0.025,-1,4,11.338],[0.025,-1,16,11.338],[0.025,1,16,11.338],[0.045,1,1,11.338],[0.045,-1,2,11.338],[0.045,-1,4,11.338],[0.045,1,4,11.338],[0.045,1,16,11.338],[0.08,1,1,11.338],[0.08,-1,2,11.338],[0.08,-1,4,11.338],[0.08,1,4,11.338]]

T = [[1.,1.,1.],[1.,1.,0.],[1.,0.,0.]]

angularsymmvec = []

for element in V 
    for types in T
        symmfunc = AngularType3{Float64}(element[1],element[2],element[3],11.338,types)
        push!(angularsymmvec,symmfunc)
    end
end

total_symm_vec = vcat(radsymmvec,angularsymmvec)

88-element Vector{Any}:
 RadialType2{Float64}(0.001, 11.338, [1.0, 1.0])
 RadialType2{Float64}(0.001, 11.338, [1.0, 0.0])
 RadialType2{Float64}(0.02, 11.338, [1.0, 1.0])
 RadialType2{Float64}(0.02, 11.338, [1.0, 0.0])
 RadialType2{Float64}(0.035, 11.338, [1.0, 1.0])
 RadialType2{Float64}(0.035, 11.338, [1.0, 0.0])
 RadialType2{Float64}(0.1, 11.338, [1.0, 1.0])
 RadialType2{Float64}(0.1, 11.338, [1.0, 0.0])
 RadialType2{Float64}(0.4, 11.338, [1.0, 1.0])
 RadialType2{Float64}(0.4, 11.338, [1.0, 0.0])
 ⋮
 AngularType3{Float64}(0.08, -1.0, 2.0, 11.338, [1.0, 1.0, 1.0])
 AngularType3{Float64}(0.08, -1.0, 2.0, 11.338, [1.0, 1.0, 0.0])
 AngularType3{Float64}(0.08, -1.0, 2.0, 11.338, [1.0, 0.0, 0.0])
 AngularType3{Float64}(0.08, -1.0, 4.0, 11.338, [1.0, 1.0, 1.0])
 AngularType3{Float64}(0.08, -1.0, 4.0, 11.338, [1.0, 1.0, 0.0])
 AngularType3{Float64}(0.08, -1.0, 4.0, 11.338, [1.0, 0.0, 0.0])
 AngularType3{Float64}(0.08, 1.0, 4.0, 11.338, [1.0, 1.0, 1.0])
 AngularType3{Float64}(0.08, 1.0, 4.0, 

Ready to start workshopping:

Considering the cutoff radius of every symmetry function is identical, we can calculate an f_cut matrix. 

In [3]:
f_mat = cutoff_function.(sqrt.(dis2mat),Ref(total_symm_vec[1].r_cut))

55×55 Matrix{Float64}:
 1.0        0.649134    0.649134    …  0.21115     0.21115     0.0959383
 0.649134   1.0         0.617699       0.0         0.0         0.0
 0.649134   0.617699    1.0            0.00208061  0.0912928   0.0
 0.649134   0.617699    0.617699       0.0         0.00208061  0.0
 0.649134   0.617699    0.223352       0.00208061  0.0         0.0
 0.649134   0.617699    0.223352    …  0.0912928   0.00208061  0.0
 0.649134   0.617699    0.617699       0.0912928   0.0912928   0.0
 0.649134   0.223352    0.617699       0.0912928   0.355548    0.185434
 0.649134   0.223352    0.223352       0.0912928   0.0912928   0.185434
 0.649134   0.223352    0.0889637      0.355548    0.0912928   0.185434
 ⋮                                  ⋱                          
 0.0959383  0.0         0.0            0.623382    0.164805    0.0613471
 0.21115    0.00208061  0.0912928      0.608792    0.608792    0.164805
 0.0959383  0.0         0.185434       0.164805    0.623382    0.0613471
 0.2

In [4]:
calc_one_symm_val(r2_ij,fc_ij,eta) = ifelse(fc_ij!=0. && fc_ij!=1., fc_ij*exp(-eta*r2_ij), 0.)

function calc_symm_function(positions,dist2_matrix,fc_matrix,index,symmfunc::RadialType2)
    
    eta=symmfunc.eta
    g_vec = calc_one_symm_val.(dist2_matrix[:,index],fc_matrix[:,index],eta)
    return sum(g_vec)
end


calc_symm_function (generic function with 1 method)

In [5]:
function calc_one_symm_val(position1,position2,position3,r2_ij,r2_ik,r2_jk,f_ij,f_ik,f_jk,η,λ,ζ)
        θ = angular_measure(position1,position2,position3,r2_ij,r2_ik)
    
        g= (1+λ*θ)^ζ * exp(-η*(r2_ij+r2_ik+r2_jk)) * f_ij * f_ik * f_jk
        
    return g
end

function calc_symm_function(positions,dis2_mat,fc_mat,index,symmfunc::AngularType3)
    N = length(positions)
    g = 0.
    η,λ,ζ = symmfunc.eta,symmfunc.lambda,symmfunc.zeta
    if symmfunc.type_vec == [1.,1.,1.]
        for j=(1:N)
            if j != index
                for k  = (1:j-1)
                    if k!= index
                    #ind = Int((j^2 - 3j)/2 + 1 + k)
                        g+= calc_one_symm_val(
                            positions[index],positions[j],positions[k],dis2_mat[index,j],dis2_mat[index,k],dis2_mat[j,k],fc_mat[index,j],fc_mat[index,k],fc_mat[j,k] ,η,λ,ζ
                            ) 
                    end            
                end
            end
        end
    end
   
    return 2^(1-ζ)*g
end

calc_symm_function (generic function with 2 methods)

In [6]:
function symm_functions(positions,dis2_mat,fc_mat,index,symm_func_vec)
    G_vec = calc_symm_function.(Ref(positions),Ref(dis2_mat),Ref(fc_mat),Ref(index),symm_func_vec)
    return G_vec
end
@benchmark symm_functions($positions,$dis2mat,$f_mat,rand(1:55),$total_symm_vec)

BenchmarkTools.Trial: 5421 samples with 1 evaluation.
 Range (min … max):  885.960 μs …  2.600 ms  ┊ GC (min … max): 0.00% … 64.60%
 Time  (median):     924.177 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   921.476 μs ± 36.271 μs  ┊ GC (mean ± σ):  0.07% ±  1.24%

                  ▂ ▃▁▁         ▃▇▆█▃▁                          
  ▂▅▄█▆▄▄▄▃▃▃▃▃▃▄▇█████▇▆▅▅▄▅▅▅▆██████▆▆▅▄▄▄▄▃▄▃▆▄▄▃▂▃▂▂▂▂▂▁▁▁ ▄
  886 μs          Histogram: frequency by time          963 μs <

 Memory estimate: 23.25 KiB, allocs estimate: 209.

In [7]:
@benchmark calc_symm_function($positions,$dis2mat,$f_mat,rand(1:55),$total_symm_vec[11])

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  31.211 μs …  57.567 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     32.026 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   32.169 μs ± 767.241 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▆▅▅▄▂        ▁▇██▇▅▃        ▁▅▅▅▅▄          ▁▂▂▂▁           ▂
  ██████▇▅▄▂▅▄▅▅█████████▇▇▇▇▇▇█████████▆█████████████▇▇▆▇▅▇▅▆ █
  31.2 μs       Histogram: log(frequency) by time        34 μs <

 Memory estimate: 96 bytes, allocs estimate: 2.

In [8]:
@benchmark calc_symmetry_function($positions,$dis2mat,rand(1:55),$total_symm_vec[11])

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  21.512 μs … 953.937 μs  ┊ GC (min … max): 0.00% … 92.81%
 Time  (median):     27.488 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   28.964 μs ±  19.413 μs  ┊ GC (mean ± σ):  1.43% ±  2.12%

               ▄▇███▆▃         ▁▁                               
  ▂▇▇▇█▆▅▃▂▁▂▃▆███████▇▄▃▂▂▃▄▅█████▇▆▅▃▃▂▂▂▂▂▃▂▂▃▂▂▂▂▁▁▁▁▁▁▁▁▁ ▃
  21.5 μs         Histogram: frequency by time         41.4 μs <

 Memory estimate: 23.39 KiB, allocs estimate: 4.

More important here is to gauge how slow it is to calculate the total symmetry vector for each atom

In [9]:
function total_total_symm(positions,dis2_mat,fc_mat,symm_func_vec)
    g_tot=[]
    for index in eachindex(positions)
        G_vec = symm_functions(positions,dis2mat,f_mat,rand(1:55),total_symm_vec)
        push!(g_tot,G_vec)
    end
    return g_tot
end
@benchmark total_total_symm($positions,$dis2mat,$f_mat,$total_symm_vec)

BenchmarkTools.Trial: 98 samples with 1 evaluation.
 Range (min … max):  49.724 ms …  54.046 ms  ┊ GC (min … max): 0.00% … 3.50%
 Time  (median):     51.230 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   51.188 ms ± 658.175 μs  ┊ GC (mean ± σ):  0.08% ± 0.50%

                 ▁ ▁▃    ▄▁ █▃▆▆   ▁                            
  ▄▁▄▄▄▆▄▁▁▆▁▆▆▇▆█▆██▄▆▆▆██▄████▇▇▆█▁▁▇▄▄▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  49.7 ms         Histogram: frequency by time         53.4 ms <

 Memory estimate: 1.25 MiB, allocs estimate: 11499.

In [10]:
total_total_symm(positions,dis2mat,f_mat,total_symm_vec)

55-element Vector{Any}:
 [11.966607067163583, 11.966607067163583, 6.362235446694212, 6.362235446694212, 4.143106864339767, 4.143106864339767, 0.8571680646436799, 0.8571680646436799, 0.0011760266338472004, 0.0011760266338472004  …  0.0, 0.005002047517575896, 0.0, 0.0, 0.0005983507129227555, 0.0, 0.0, 0.021924952304756905, 0.0, 0.0]
 [5.699536975224264, 5.699536975224264, 2.9415593759647933, 2.9415593759647933, 1.9155542688220328, 1.9155542688220328, 0.41077108438095306, 0.41077108438095306, 0.0005573982223337895, 0.0005573982223337895  …  0.0, 0.0016597483689902927, 0.0, 0.0, 0.00013714023992727955, 0.0, 0.0, 0.007651868031824278, 0.0, 0.0]
 [7.455371452215095, 7.455371452215095, 3.928878421937883, 3.928878421937883, 2.545960542068822, 2.545960542068822, 0.5083986414795432, 0.5083986414795432, 0.0005285363551481289, 0.0005285363551481289  …  0.0, 0.0019216479933303451, 0.0, 0.0, 0.0001820362393914674, 0.0, 0.0, 0.010500880315042017, 0.0, 0.0]
 [7.455371452720861, 7.455371452720861, 3.92

# Stating the obvious

There are two components to this: firstly, it may be faster to implicitly remove the double calculation of triples by assigning them immediately. That is, go over symmetry functions rather than atoms and assign as we go, reducing the calculation from $55\times \sum_{j=1}^{54} j $ to $\sum_{i=1}^{55} \sum_{j=i+1}^{55}j $ This reduces the time of total calculation, but not by enough. We can further speed this up by working on the calculation of the symmetry function itself: this may require more thought. 

In [11]:
function init_symm_vecs(dist2_mat,total_symm_vec)
    g_mat=zeros(length(total_symm_vec),size(dist2_mat)[1])
    return g_mat 
end
g_mat = init_symm_vecs(dis2mat,total_symm_vec)
length(g_mat[2,:])

55

In [12]:
length(eachcol(g_mat))

55

In [13]:
function calc_symm_vals!(positions,dist2_mat,f_mat,g_vec,symm_func::RadialType2)
    N=length(g_vec)
    if symm_func.type_vec == [1.,1.]
        
        for atomindex in eachindex(g_vec)
            for index2 in (atomindex+1):N
                g_val =  calc_one_symm_val(dist2_mat[atomindex,index2],f_mat[atomindex,index2],symm_func.eta)
                g_vec[atomindex] +=g_val 
                g_vec[index2] += g_val
            end
        end
    else
        g_vec = zeros(N)
    end

    return g_vec
end

function calc_symm_vals!(positions,dist2_mat,f_mat,g_vec,symm_func::AngularType3)
    N = length(g_vec)
    η,λ,ζ = symm_func.eta,symm_func.lambda,symm_func.zeta
    if symm_func.type_vec == [1.,1.,1.]
        for atomindex in eachindex(g_vec)
            for index2 in (atomindex+1):N
                for index3 in (index2+1):N

                    g_val=calc_one_symm_val(positions[atomindex],positions[index2],positions[index3],dist2_mat[atomindex,index2],dist2_mat[atomindex,index3],dist2_mat[index2,index3],f_mat[atomindex,index2],f_mat[atomindex,index3],f_mat[index2,index3],η,λ,ζ)

                    g_vec[atomindex] += g_val
                    g_vec[index2] += g_val
                    g_vec[index3] += g_val
                end
            end
        end
    else
        g_vec = zeros(N)
    end


    return 2.0^(1-ζ)*g_vec
end

calc_symm_vals! (generic function with 2 methods)

In [14]:
function total_symm_calc(positions,dist2_mat,f_mat,total_symm_vec)
    g_mat = init_symm_vecs(dist2_mat,total_symm_vec)
     for g_index in eachindex(total_symm_vec)
        g_mat[g_index,:] = calc_symm_vals!(positions,dist2_mat,f_mat,g_mat[g_index,:],total_symm_vec[g_index])
    end

    return g_mat
end

total_symm_calc (generic function with 1 method)

In [15]:
println(total_symm_calc(positions,dis2mat,f_mat,total_symm_vec)[:,1])

[14.640666117305631, 0.0, 7.169783183727494, 0.0, 4.524213051903175, 0.0, 0.9725333082013506, 0.0, 0.0017948851872611987, 0.0, 35.92620108172102, 0.0, 0.0, 5.982413095508248, 0.0, 0.0, 10.0970998508329, 0.0, 0.0, 4.365675475640729, 0.0, 0.0, 6.09854579227037, 0.0, 0.0, 2.589615260424904, 0.0, 0.0, 12.270318492668707, 0.0, 0.0, 8.120072111423502, 0.0, 0.0, 1.296182793299763, 0.0, 0.0, 0.4555203136216524, 0.0, 0.0, 0.07521609278063308, 0.0, 0.0, 1.3389609524097035, 0.0, 0.0, 3.4140223846552784, 0.0, 0.0, 2.5901077848191956, 0.0, 0.0, 0.1607864821335121, 0.0, 0.0, 0.02191855766980976, 0.0, 0.0, 0.30798801672233045, 0.0, 0.0, 0.7308446089952683, 0.0, 0.0, 0.09696288396523911, 0.0, 0.0, 0.022012975865658216, 0.0, 0.0, 0.31164757413688454, 0.0, 0.0, 0.03673259165919136, 0.0, 0.0, 0.0650435502826457, 0.0, 0.0, 0.007425477011199372, 0.0, 0.0, 0.0009478536451217326, 0.0, 0.0, 0.02544139990466739, 0.0, 0.0]


In [16]:
@benchmark total_symm_calc($positions,$dis2mat,$f_mat,$total_symm_vec)

BenchmarkTools.Trial: 290 samples with 1 evaluation.
 Range (min … max):  16.981 ms …  18.380 ms  ┊ GC (min … max): 0.00% … 5.14%
 Time  (median):     17.311 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   17.301 ms ± 173.894 μs  ┊ GC (mean ± σ):  0.02% ± 0.30%

            █ ▂ ▂  ▁    ▃     ▄    ▁▁  ▄▂▁                      
  ▃▁▃▄▄▄▆▆▆██████▅▃█▄▇▄▄█▅▄▄▅▆████▇███▇███▄▆▄▆▇▄▄▅▅▅▄▃▃▁▄▄▃▁▁▄ ▄
  17 ms           Histogram: frequency by time         17.7 ms <

 Memory estimate: 154.12 KiB, allocs estimate: 401.